In [1]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from datetime import datetime, timedelta
from datetime import datetime
from datetime import timezone
from google.colab import drive
import os
import datetime

#drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 16.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Set your parameters
redirect_uri = 'https://jwambui.github.io/portifolio/'
client_id = '7ea7739f0e1d4c4e892b38f688f2a78f'
client_secret = 'dd2ac16002ad4b2f972a30f748322c29'

# Initialize SpotifyOAuth object
sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope='user-read-recently-played')

# Get authorization URL
auth_url = sp_oauth.get_authorize_url()

# Print the authorization URL and ask the user to open it in their browser and authorize the app
print("Please go to this URL and authorize access: \n" + auth_url)

# Once authorized, the user will be redirected to the redirect URI with the authorization code in the URL.
# Extract the authorization code from the URL manually and input it below.
code = input("Enter the authorization code: ")

Please go to this URL and authorize access: 
https://accounts.spotify.com/authorize?client_id=7ea7739f0e1d4c4e892b38f688f2a78f&response_type=code&redirect_uri=https%3A%2F%2Falexochoki.github.io%2Fportifolio%2F&scope=user-read-recently-played
Enter the authorization code: AQDDlYgxSTlSdvkdhbopnXDYCw15J4dmBeJi6bB6shTvXLUlobr1NHOI_BO0htDJs0hhY1KCa4eGtzj7jSxk4UCY978DbSBEbzWqnA6pApRJidZbMiX9doDEl9-LEhksdUhabMtp1H5ryQjDc4hmy0tDrODeWyEp-1dkfrxv80H8VP9kL09Ac-wNZh3wX5hY9MuSj7BjvZ3gcUulb6__YEq1ZEjOCow


In [6]:
# Exchange the authorization code for an access token
token_info = sp_oauth.get_access_token(code)

# Initialize Spotipy object with the access token
sp = spotipy.Spotify(auth=token_info['access_token'])

<ipython-input-6-1db8bd57f8f3>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [7]:
# Get all played tracks (handle pagination)
all_played_tracks = []
results = sp.current_user_recently_played(limit=50)

while results['items']:
    all_played_tracks.extend(results['items'])
    results = sp.current_user_recently_played(limit=50, after=results['cursors']['after'])

# Create a DataFrame to store the results
songs_df = pd.DataFrame(columns=['Track Name', 'Artist(s)', 'Album', 'Release Date', 'Duration', 'Played At'])
# Assuming 'all_played_tracks' is a list of dictionaries
data = []
for track in all_played_tracks:
    track_name = track['track']['name']
    artists = ', '.join([artist['name'] for artist in track['track']['album']['artists']])
    album_name = track['track']['album']['name']
    release_date = track['track']['album']['release_date']
    duration_ms = track['track']['duration_ms']
    duration_min_sec = divmod(duration_ms / 1000, 60)
    duration = f"{int(duration_min_sec[0])}:{int(duration_min_sec[1]):02d}"
    played_at = track['played_at']
    songs_df = {}
    songs_df['Track Name'] = track_name
    songs_df['Artist(s)'] = artists
    songs_df['Album'] = album_name
    songs_df['Release Date'] = release_date
    songs_df['Duration'] = duration
    songs_df['Played At'] = played_at
    data.append(songs_df)

# Create a new DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df=songs_df

# Extract relevant information and append to DataFrame
data_to_append = []
for track in all_played_tracks:
    track_name = track['track']['name']
    artists = ', '.join([artist['name'] for artist in track['track']['artists']])
    album_name = track['track']['album']['name']
    release_date = track['track']['album']['release_date']
    duration_ms = track['track']['duration_ms']
    duration_min_sec = divmod(duration_ms / 1000, 60)
    duration = f"{int(duration_min_sec[0])}:{int(duration_min_sec[1]):02d}"
    played_at = track['played_at']

    data_to_append.append({
        'Track Name': track_name,
        'Artist(s)': artists,
        'Album': album_name,
        'Release Date': release_date,
        'Duration': duration,
        'Played At': played_at
    })

songs_df = pd.DataFrame(data_to_append)
songs_df.head()

,Track Name,Artist(s),Album,Release Date,Duration,Played At
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,3:20,2024-08-22T15:52:53.658Z
1,I Feel It Coming,"The Weeknd, Daft Punk",Starboy,2016-11-25,4:29,2024-08-22T15:49:35.785Z
2,Starboy,"The Weeknd, Daft Punk",Starboy,2016-11-25,3:50,2024-08-22T15:45:08.330Z
3,Heartless,The Weeknd,After Hours,2020-03-20,3:18,2024-08-22T15:41:13.055Z
4,Ordinary Life,The Weeknd,Starboy,2016-11-24,3:41,2024-08-22T15:37:50.421Z


In [8]:
# add new column record_id by taking the Played At convert it to string and removing the space and colon, then concatenate it with the trackname. Also remove the dash (-) in the date

# Add a new column 'record_id' to the DataFrame
songs_df['record_id'] = songs_df['Played At'].astype(str).str.replace(' ', '').str.replace(':', '') + songs_df['Track Name']

# add new column year from 'Played At' by extracting year, show top 3 records

# Extract year from 'Played At' and add it as a new column
songs_df['Year'] = pd.to_datetime(songs_df['Played At']).dt.year

# add new column month from 'Played At' by extracting month name just 3 letters, show top 3 records

# Extract month name and first 3 letters only
songs_df['Month'] = pd.to_datetime(songs_df['Played At']).dt.strftime('%b')

#  add new column month year from 'Played At' by extracting month and  year, show top 3 records

# Extract month and year from 'Played At' and add it as a new column
songs_df['Month Year'] = pd.to_datetime(songs_df['Played At']).dt.strftime('%b %Y')

# add new column date by extracting date from 'Played At'

# Extract date from 'Played At' and add it as a new column
songs_df['Date'] = pd.to_datetime(songs_df['Played At']).dt.date

# add new column hour from 'Played At' by extracting hour from 'Played At', show top 3 records

songs_df['Hour'] = pd.to_datetime(songs_df['Played At']).dt.hour

# add column day of the week by extracting the day name from PLayed at, should be just 3 letters

songs_df['Day'] = pd.to_datetime(songs_df['Played At']).dt.strftime('%a')

# add column day date by taking the date of the month from played at

songs_df['Day Date'] = pd.to_datetime(songs_df['Played At']).dt.day

#Add new column 'Minutes' by converting Duration to minutes in in point form

songs_df['Minutes Listened'] = songs_df['Duration'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1])/60)

# Add nwew column 'Hours Listened' by converting Minutes to hours

songs_df['Hours Listened'] = songs_df['Minutes Listened'] / 60


# Change the columns names to ensure each word of the column name starts with capital letter

songs_df.columns = songs_df.columns.str.title()

# reorder the columns so that we have them: Record_Id, Trackname, Artist(S), Album, Duration, Played at, Hour, Date, Month, Month year, Year

songs_df = songs_df[['Record_Id', 'Track Name', 'Artist(S)', 'Album', 'Duration', 'Minutes Listened', 'Hours Listened', 'Played At', 'Hour', 'Day', 'Day Date', 'Date', 'Month', 'Month Year', 'Year']]
songs_df.head(5)

,Record_Id,Track Name,Artist(S),Album,Duration,Minutes Listened,Hours Listened,Played At,Hour,Day,Day Date,Date,Month,Month Year,Year
0,2024-08-22T155253.658ZBlinding Lights,Blinding Lights,The Weeknd,After Hours,3:20,3.333333,0.055556,2024-08-22T15:52:53.658Z,15,Thu,22,2024-08-22,Aug,Aug 2024,2024
1,2024-08-22T154935.785ZI Feel It Coming,I Feel It Coming,"The Weeknd, Daft Punk",Starboy,4:29,4.483333,0.074722,2024-08-22T15:49:35.785Z,15,Thu,22,2024-08-22,Aug,Aug 2024,2024
2,2024-08-22T154508.330ZStarboy,Starboy,"The Weeknd, Daft Punk",Starboy,3:50,3.833333,0.063889,2024-08-22T15:45:08.330Z,15,Thu,22,2024-08-22,Aug,Aug 2024,2024
3,2024-08-22T154113.055ZHeartless,Heartless,The Weeknd,After Hours,3:18,3.300000,0.055000,2024-08-22T15:41:13.055Z,15,Thu,22,2024-08-22,Aug,Aug 2024,2024
4,2024-08-22T153750.421ZOrdinary Life,Ordinary Life,The Weeknd,Starboy,3:41,3.683333,0.061389,2024-08-22T15:37:50.421Z,15,Thu,22,2024-08-22,Aug,Aug 2024,2024


In [9]:
# drop the duplicate records in songs_df and print the number of duplcates dropped

# Drop duplicate rows from songs_df
songs_df.drop_duplicates(inplace=True)

# Calculate the number of duplicates dropped
num_duplicates = len(songs_df.index) - len(songs_df.drop_duplicates().index)

# Print the number of duplicates dropped
print(f"Number of duplicates dropped: {num_duplicates}")

Number of duplicates dropped: 0


In [10]:
# Get the current date and time in UTC
utc_now = datetime.datetime.now(timezone.utc)

# Convert UTC to East Africa Standard Time (EAT)
eat_now = utc_now.astimezone(timezone(timedelta(hours=3)))

# Get the current date and time and format it appropriately
current_date_time = eat_now.strftime("%Y-%m-%d_%H:%M:%S")

# Save the DataFrame to a CSV file with the current date and time appended to the filename
songs_df.to_csv(f'/content/drive/MyDrive/Spotify Data Files/daily_loads/spotify_history_{current_date_time}.csv', index=False)

In [11]:
# scan  this folder increamental load folder and identify the latest file based on the last modified date and provide its full path also

import os
import datetime

def get_latest_file(folder):
  """
  This function scans a folder and returns the latest file based on the last modified date.

  Args:
      folder (str): The path to the folder to scan.

  Returns:
      str: The full path to the latest file.
  """

  latest_file = None
  latest_mtime = None

  for filename in os.listdir(folder):
    filepath = os.path.join(folder, filename)

    # Check if the file is a regular file and not a directory.
    if os.path.isfile(filepath):
      mtime = datetime.datetime.fromtimestamp(os.path.getmtime(filepath))

      # Update the latest file if the current file has a more recent modification time.
      if not latest_file or mtime > latest_mtime:
        latest_file = filepath
        latest_mtime = mtime

  return latest_file

# Get the latest file in the specified folder.
latest_file = get_latest_file('/content/drive/MyDrive/Spotify Data Files/daily_loads')

# Print the full path to the latest file.
print(latest_file)

/content/drive/MyDrive/Spotify Data Files/daily_loads/spotify_history_2024-08-22_21:01:41.csv


In [12]:
# Load dataframes
ipop_df_path = '/content/drive/MyDrive/Spotify Data Files/Incremental/8_incremental.csv'
daily_df_path = latest_file

ipop_df = pd.read_csv(ipop_df_path)
daily_df = pd.read_csv(daily_df_path)

# Check for records in daily_df that don't exist in ipop_df
new_records = daily_df[~daily_df['Record_Id'].isin(ipop_df['Record_Id'])]

# If there are new records, append them to ipop_df using pd.concat()
if not new_records.empty:
    ipop_df = pd.concat([ipop_df, new_records], ignore_index=True) # Use pd.concat() instead of .append()
    num_new_records = len(new_records)
    print(f"{num_new_records} new records added.")
else:
    print("No new records were added.")

# Save the combined dataframe to a new file
output_path = '/content/drive/MyDrive/Spotify Data Files/Incremental/8_incremental.csv'
ipop_df.to_csv(output_path, index=False)

50 new records added.


In [13]:
#get the record count
len(ipop_df)

25068

In [14]:
# find the top 5 days(Date) I listened to most tracks

top_5_days = ipop_df.groupby('Date')['Track Name'].count().sort_values(ascending=False)

top_5_days.head()


,Track Name
Date,
2023-10-14,216
2023-11-23,209
2024-01-19,197
2023-11-01,192
2021-01-07,189


#Making The Minutes Column uniform

In [ ]:
# Add a column 'Mins' which uses this logic:
# If the column value in the column does not have colon (:) then divide it with 60000, if it does have a column, take the value after the colon divide it by 60 then add it to value before the column. Implement it using the dataframe df

import math

def add_mins_column(df):
  """
  This function adds a new column 'Mins' to the dataframe df.
  The value in the 'Mins' column is calculated as follows:
  - If the value in the specified column does not have a colon (:), divide it by 60000.
  - If the value in the specified column has a colon, split it at the colon, convert the value after the colon to minutes, and add it to the value before the colon.

  Args:
      df (pd.DataFrame): The dataframe to which the new column will be added.

  Returns:
      pd.DataFrame: The dataframe with the new 'Mins' column added.
  """

  def calculate_mins(value):
    if ':' in value:
      minutes = int(value.split(':')[0]) + int(value.split(':')[1])/60
    else:
      minutes = int(value) / 60000
    return minutes

  df['Minutes Listened'] = df['Duration'].apply(calculate_mins)
  return df

# Add the 'Mins' column to the dataframe
df = add_mins_column(df)

# Display the last 5 rows of the dataframe
df.head()


,Record_Id,Track Name,Artist(S),Album,Duration,Played At,Hour,Day,Day Date,Date,Month,Month Year,Year,Record_Id.1,Mins,Minutes Listened
0,2020-12-24110108I'm Yours,I'm Yours,Jason Mraz,We Sing. We Dance. We Steal Things.,60688,2020-12-24 11:01:08,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,1.011467,1.011467
1,2020-12-24110215FourFiveSeconds,FourFiveSeconds,Rihanna,FourFiveSeconds,64187,2020-12-24 11:02:15,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,1.069783,1.069783
2,2020-12-24111026Animals,Animals,Martin Garrix,Animals,1006,2020-12-24 11:10:26,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,0.016767,0.016767
3,2020-12-24133334Animals,Animals,Martin Garrix,Animals,160060,2020-12-24 13:33:34,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,2.667667,2.667667
4,2020-12-24133957Champion,Champion,Nicki Minaj,Pink Friday: Roman Reloaded The Re-Up,296040,2020-12-24 13:39:57,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,4.934000,4.934000


In [ ]:
# add column Hours Listened from Minutes Listened/60

df['Hours Listened'] = df['Minutes Listened'] / 60
df.head()


,Record_Id,Track Name,Artist(S),Album,Duration,Played At,Hour,Day,Day Date,Date,Month,Month Year,Year,Record_Id.1,Mins,Minutes Listened,Hours Listened
0,2020-12-24110108I'm Yours,I'm Yours,Jason Mraz,We Sing. We Dance. We Steal Things.,60688,2020-12-24 11:01:08,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,1.011467,1.011467,0.016858
1,2020-12-24110215FourFiveSeconds,FourFiveSeconds,Rihanna,FourFiveSeconds,64187,2020-12-24 11:02:15,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,1.069783,1.069783,0.017830
2,2020-12-24111026Animals,Animals,Martin Garrix,Animals,1006,2020-12-24 11:10:26,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,0.016767,0.016767,0.000279
3,2020-12-24133334Animals,Animals,Martin Garrix,Animals,160060,2020-12-24 13:33:34,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,2.667667,2.667667,0.044461
4,2020-12-24133957Champion,Champion,Nicki Minaj,Pink Friday: Roman Reloaded The Re-Up,296040,2020-12-24 13:39:57,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020,NaN,4.934000,4.934000,0.082233


In [ ]:
df = df[['Record_Id', 'Track Name', 'Artist(S)', 'Album', 'Duration', 'Minutes Listened', 'Hours Listened', 'Played At', 'Hour', 'Day', 'Day Date', 'Date', 'Month', 'Month Year', 'Year']]
df.head(5)

,Record_Id,Track Name,Artist(S),Album,Duration,Minutes Listened,Hours Listened,Played At,Hour,Day,Day Date,Date,Month,Month Year,Year
0,2020-12-24110108I'm Yours,I'm Yours,Jason Mraz,We Sing. We Dance. We Steal Things.,60688,1.011467,0.016858,2020-12-24 11:01:08,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020
1,2020-12-24110215FourFiveSeconds,FourFiveSeconds,Rihanna,FourFiveSeconds,64187,1.069783,0.017830,2020-12-24 11:02:15,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020
2,2020-12-24111026Animals,Animals,Martin Garrix,Animals,1006,0.016767,0.000279,2020-12-24 11:10:26,11,Thu,24.0,2020-12-24,Dec,Dec 2020,2020
3,2020-12-24133334Animals,Animals,Martin Garrix,Animals,160060,2.667667,0.044461,2020-12-24 13:33:34,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020
4,2020-12-24133957Champion,Champion,Nicki Minaj,Pink Friday: Roman Reloaded The Re-Up,296040,4.934000,0.082233,2020-12-24 13:39:57,13,Thu,24.0,2020-12-24,Dec,Dec 2020,2020


In [ ]:
len(df)

6

In [ ]:
output_path = '/content/drive/MyDrive/Spotify Data Files/Incremental/8_incremental.csv'
df.to_csv(output_path, index=False)

# Compare records with the Files on Laptop & add them

In [19]:
output_path = '/content/drive/MyDrive/Spotify Data Files/Incremental/8_incremental.csv'
laptop_file = '/content/drive/MyDrive/Spotify Data Files/Incremental/full_listening_history.csv'
offline_data = pd.read_csv(laptop_file)
drive_data = pd.read_csv(output_path)


In [20]:
# check the records that exist in laptop file, if there are records in offline_data dataframe that are not in drive_data dataframe, add them and save to the output_path. Also print the number of records that were missing and were added

# Identify records that are in offline_data but not in drive_data
missing_records = offline_data[~offline_data['Record_Id'].isin(drive_data['Record_Id'])]

# Add missing records to drive_data using concat
drive_data = pd.concat([drive_data, missing_records], ignore_index=True)

# Print the number of records that were added
num_added_records = len(missing_records)
print(f"{num_added_records} records were missing from the drive file and have been added.")

# Save the combined dataframe to the output file
drive_data.to_csv(output_path, index=False)

4174 records were missing from the drive file and have been added.


In [18]:
len(drive_data)

25354

# For Offline - Jupyter Notebook

In [ ]:
# check the records that exist in drive_file_path, if there are records in drive_data dataframe that are not in offline_data dataframe, add them and save to the laptop_file. Also print the number of records that were missing and were added

# Identify records that are in drive_data but not in offline_data
missing_records = drive_data[~drive_data['Record_Id'].isin(offline_data['Record_Id'])]

# Add missing records to offline_data
offline_data = offline_data.append(missing_records, ignore_index=True)

# Print the number of records that were added
num_added_records = len(missing_records)
print(f"{num_added_records} records were missing from the laptop file and have been added.")

# Save the combined dataframe to the laptop file
offline_data.to_csv(laptop_file, index=False)
